In [1]:
!pip install booknlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00
  Created wheel for booknlp: filename=booknlp-1.0.7-py3-none-any.whl size=2420743 sha256=4c1c01104b5f2df1f927230c8c889edc2cccbea9eae777b54c8e0ff5c5ea89d2
  Stored in directory: /root/.cache/pip/wheels/bb/f5/b4/2ee6b317da6491ff01b742be603af81ceea156427f1f4be38a
Successfully built booknlp


In [ ]:
!unzip '/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/scifiTVshows.zip'

Archive:  /content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/scifiTVshows.zip
   creating: OriginalStoriesSeparated/
  inflating: OriginalStoriesSeparated/Babylon5.txt  
  inflating: OriginalStoriesSeparated/DoctorWhoSpinoffs.txt  
  inflating: OriginalStoriesSeparated/DoctorWho.txt  
  inflating: OriginalStoriesSeparated/Fringe.txt  
  inflating: OriginalStoriesSeparated/Farscape.txt  
  inflating: OriginalStoriesSeparated/Futurama.txt  
  inflating: OriginalStoriesSeparated/Stargate.txt  
  inflating: OriginalStoriesSeparated/StarTrek.txt  
  inflating: OriginalStoriesSeparated/StarWarsBooks.txt  
  inflating: OriginalStoriesSeparated/StarWarsRebels.txt  
  inflating: OriginalStoriesSeparated/XFiles.txt  
  inflating: scifi-test.txt          
  inflating: scifi-train.txt         
  inflating: scifi-val.txt           


In [2]:
from booknlp.booknlp import BookNLP

model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big"
	}
	
booknlp=BookNLP("en", model_params)

using device cpu
{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'big'}
downloading entities_google_bert_uncased_L-6_H-768_A-12-v1.0.model
downloading coref_google_bert_uncased_L-12_H-768_A-12-v1.0.model
downloading speaker_google_bert_uncased_L-12_H-768_A-12-v1.0.1.model


--- startup: 47.491 seconds ---


In [3]:
# Input file to process
input_file="/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/SampleBabylon.txt"

# Output directory to store resulting files in
output_directory="test_booknlp/"

# File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
book_id="sample_babylon"

booknlp.process(input_file, output_directory, book_id)

--- spacy: 0.961 seconds ---
--- entities: 26.277 seconds ---
--- quotes: 0.009 seconds ---
--- attribution: 1.104 seconds ---
--- name coref: 0.013 seconds ---
--- coref: 22.531 seconds ---
--- TOTAL (excl. startup): 50.917 seconds ---, 2951 words


In [4]:
import json 
# reading the data from the file
with open('/content/test_booknlp/sample_babylon.book') as f:
    data = f.read()
  
print("Data type before reconstruction : ", type(data))
      
# reconstructing the data as a dictionary
js = json.loads(data)

Data type before reconstruction :  <class 'str'>


In [ ]:
print(js)

{'characters': [{'agent': [{'w': 'receives', 'i': 40063}, {'w': 'command', 'i': 40149}, {'w': 'arrives', 'i': 40373}, {'w': 'explains', 'i': 40497}, {'w': 'had', 'i': 40520}, {'w': 'hopes', 'i': 40562}, {'w': 'mentions', 'i': 40606}, {'w': 'plans', 'i': 40610}, {'w': 'gives', 'i': 40627}, {'w': 'arrives', 'i': 40723}, {'w': 'begins', 'i': 40727}, {'w': 'expresses', 'i': 40736}, {'w': 'assumes', 'i': 40859}, {'w': 'asks', 'i': 40924}, {'w': 'defends', 'i': 40945}, {'w': 'destroyed', 'i': 40991}, {'w': 'explains', 'i': 40997}, {'w': 'came', 'i': 41019}, {'w': 'arrives', 'i': 41111}, {'w': 'surrenders', 'i': 41118}, {'w': 'introduces', 'i': 41123}, {'w': 'decides', 'i': 41140}, {'w': 'finds', 'i': 41152}, {'w': 'head', 'i': 41423}, {'w': 'sound', 'i': 41427}, {'w': 'warns', 'i': 41464}, {'w': 'replies', 'i': 41482}, {'w': 'worries', 'i': 41496}, {'w': 'orders', 'i': 41532}, {'w': 'realizes', 'i': 41547}, {'w': 'orders', 'i': 41559}, {'w': 'sends', 'i': 41568}, {'w': 'explains', 'i': 41614

In [ ]:
for i in js:
  for j in js[i][0]:
    print(j)
    print(js[i][0][j])

agent
[{'w': 'receives', 'i': 40063}, {'w': 'command', 'i': 40149}, {'w': 'arrives', 'i': 40373}, {'w': 'explains', 'i': 40497}, {'w': 'had', 'i': 40520}, {'w': 'hopes', 'i': 40562}, {'w': 'mentions', 'i': 40606}, {'w': 'plans', 'i': 40610}, {'w': 'gives', 'i': 40627}, {'w': 'arrives', 'i': 40723}, {'w': 'begins', 'i': 40727}, {'w': 'expresses', 'i': 40736}, {'w': 'assumes', 'i': 40859}, {'w': 'asks', 'i': 40924}, {'w': 'defends', 'i': 40945}, {'w': 'destroyed', 'i': 40991}, {'w': 'explains', 'i': 40997}, {'w': 'came', 'i': 41019}, {'w': 'arrives', 'i': 41111}, {'w': 'surrenders', 'i': 41118}, {'w': 'introduces', 'i': 41123}, {'w': 'decides', 'i': 41140}, {'w': 'finds', 'i': 41152}, {'w': 'head', 'i': 41423}, {'w': 'sound', 'i': 41427}, {'w': 'warns', 'i': 41464}, {'w': 'replies', 'i': 41482}, {'w': 'worries', 'i': 41496}, {'w': 'orders', 'i': 41532}, {'w': 'realizes', 'i': 41547}, {'w': 'orders', 'i': 41559}, {'w': 'sends', 'i': 41568}, {'w': 'explains', 'i': 41614}, {'w': 'assumed', 

In [ ]:
print(js['characters'][0]['patient'])

[{'w': 'informs', 'i': 40078}, {'w': 'wants', 'i': 40122}, {'w': 'greets', 'i': 40381}, {'w': 'shows', 'i': 40474}, {'w': 'assigned', 'i': 40491}, {'w': 'disfavored', 'i': 40557}, {'w': 'interrupted', 'i': 40748}, {'w': 'asks', 'i': 40986}, {'w': 'introduces', 'i': 41123}, {'w': 'accuses', 'i': 41179}, {'w': 'approaches', 'i': 41192}, {'w': 'contacts', 'i': 41701}, {'w': 'assures', 'i': 41705}, {'w': 'visited', 'i': 41722}, {'w': 'reassures', 'i': 41783}, {'w': 'telling', 'i': 41786}, {'w': 'guess', 'i': 41792}, {'w': 'supposed', 'i': 41869}, {'w': 'met', 'i': 42071}, {'w': 'concerned', 'i': 42127}, {'w': 'consoles', 'i': 42365}, {'w': 'collect', 'i': 42625}, {'w': 'introduces', 'i': 42669}, {'w': 'contacts', 'i': 43299}, {'w': 'calls', 'i': 43770}, {'w': 'see', 'i': 44773}, {'w': 'assuring', 'i': 44915}, {'w': 'excuses', 'i': 45722}, {'w': 'informs', 'i': 45874}, {'w': 'asks', 'i': 45968}, {'w': 'chosen', 'i': 46080}, {'w': 'made', 'i': 46089}, {'w': 'trained', 'i': 46537}, {'w': 'bea

In [6]:
import json
from collections import Counter

In [5]:
def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

In [7]:
data=proc("/content/test_booknlp/sample_babylon.book")

In [8]:
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter

In [10]:
for character in data['characters']:
  print(character)

{'agent': [{'w': 'represent', 'i': 281}, {'w': 'receives', 'i': 365}, {'w': 'seals', 'i': 466}, {'w': 'welcoming', 'i': 535}, {'w': 'become', 'i': 563}, {'w': 'poisoned', 'i': 983}, {'w': 'arrived', 'i': 988}, {'w': 'arrives', 'i': 1064}, {'w': 'confront', 'i': 1086}, {'w': 'finds', 'i': 1099}, {'w': 'fighting', 'i': 1101}, {'w': 'asks', 'i': 1167}, {'w': 'manages', 'i': 1201}, {'w': 'believes', 'i': 1313}, {'w': 'believes', 'i': 1327}, {'w': 'tells', 'i': 1384}, {'w': 'slipped', 'i': 1399}, {'w': 'warns', 'i': 1411}, {'w': 'reveals', 'i': 1450}, {'w': 'keep', 'i': 1470}, {'w': 'reveals', 'i': 1505}, {'w': 'tells', 'i': 1536}, {'w': 'fought', 'i': 1541}, {'w': 'is', 'i': 1556}, {'w': 'account', 'i': 1578}, {'w': 'know', 'i': 1654}, {'w': 'listening', 'i': 1819}, {'w': 'explains', 'i': 1860}, {'w': 'see', 'i': 1887}, {'w': 'tells', 'i': 2010}, {'w': 'convinced', 'i': 2022}, {'w': 'tells', 'i': 2029}, {'w': 'calling', 'i': 2033}, {'w': 'form', 'i': 2043}, {'w': 'goes', 'i': 2134}, {'w': 

In [9]:
for character in data['characters']:
  # print(character)
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]

    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"
    
    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

19 81 Sinclair he/him/his

	poss	3 wing
	poss	2 mind
	poss	1 place
	poss	1 fate
	poss	1 deportation
	poss	1 voyage
	poss	1 role
	poss	1 people
	poss	1 grandfather
	poss	1 discovery

	agent	4 tells
	agent	2 believes
	agent	2 reveals
	agent	1 represent
	agent	1 receives
	agent	1 seals
	agent	1 welcoming
	agent	1 become
	agent	1 poisoned
	agent	1 arrived

	patient	3 tells
	patient	1 accuses
	patient	1 removes
	patient	1 turn
	patient	1 told
	patient	1 deported
	patient	1 leads
	patient	1 poisoned
	patient	1 pick
	patient	1 discredit

	mod	1 innocent

24 44 G'Kar he/him/his

	poss	1 ship
	poss	1 drink
	poss	1 business
	poss	1 threats
	poss	1 dream
	poss	1 assertion
	poss	1 explanation

	agent	1 made
	agent	1 reveals
	agent	1 received
	agent	1 pressured
	agent	1 meets
	agent	1 tells
	agent	1 endanger
	agent	1 thinks
	agent	1 keep
	agent	1 find

	patient	2 find
	patient	2 kill
	patient	1 tells
	patient	1 warns
	patient	1 finds
	patient	1 assaulting
	patient	1 sees
	patient	1 admonishes
	pati

In [ ]:
import tarfile
my_tar = tarfile.open('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/MovieSummaries.tar.gz')
my_tar.extractall('./CMU_Summaries')
my_tar.close()

In [12]:
# with open('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/SampleBabylon.txt','r') as file:
#   data=file.read()

# sentences = data.split('.')

In [109]:
df_sentences = pd.read_table('/content/test_booknlp/sample_babylon.tokens')
# df_sentences['sentence'] = df_sentences.groupby(['sentence_ID'])['word'].apply(lambda x: ' '.join(x)).reset_index(drop=True)
# df_sentences['start'] = 
print(df_sentences.groupby(['sentence_ID'])['token_ID_within_document'].apply(lambda x: ' '.join(str(x))))

sentence_ID
0      0             0 \n 1             1 \n 2       ...
1      1 9         1 9 \n 2 0         2 0 \n 2 1     ...
2      4 6         4 6 \n 4 7         4 7 \n 4 8     ...
3      7 7             7 7 \n 7 8             7 8 \n ...
4      1 0 8         1 0 8 \n 1 0 9         1 0 9 \n ...
                             ...                        
127    2 8 4 6         2 8 4 6 \n 2 8 4 7         2 8...
128    2 8 8 2         2 8 8 2 \n 2 8 8 3         2 8...
129    2 8 9 9         2 8 9 9 \n 2 9 0 0         2 9...
130    2 9 3 3         2 9 3 3 \n 2 9 3 4         2 9...
131    2 9 4 8         2 9 4 8 \n 2 9 4 9         2 9...
Name: token_ID_within_document, Length: 132, dtype: object


In [102]:
df_sentences['sentence'].dropna()

0      Alien envoys come to the giant space station i...
1      In the earth year 2257 , a multitude of humans...
2      Some of them are members of the station crew ,...
3      Others are ambassadors from major alien govern...
4      Still others are refugees , smugglers , busine...
                             ...                        
127    Ivanova explains why she has been avoiding Win...
128    The drugs lead Ivanova 's mother into a deep d...
129    Garibaldi has convinced Ambassador Delenn to c...
130    Sinclair watches election returns , showing a ...
131                                              < EOS >
Name: sentence, Length: 132, dtype: object

In [110]:
df_sentences.head(-5)

,paragraph_ID,sentence_ID,token_ID_within_sentence,token_ID_within_document,word,lemma,byte_onset,byte_offset,POS_tag,fine_POS_tag,dependency_relation,syntactic_head_ID,event
0,0,0,0,0,Alien,alien,0,5,ADJ,JJ,amod,1,O
1,0,0,1,1,envoys,envoy,6,12,NOUN,NNS,nsubj,2,O
2,0,0,2,2,come,come,13,17,VERB,VBP,ROOT,2,EVENT
3,0,0,3,3,to,to,18,20,ADP,IN,prep,2,O
4,0,0,4,4,the,the,21,24,DET,DT,det,7,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2941,0,130,8,2941,for,for,15540,15543,ADP,IN,prep,2940,O
2942,0,130,9,2942,the,the,15544,15547,DET,DT,det,2944,O
2943,0,130,10,2943,incumbent,incumbent,15548,15557,ADJ,JJ,amod,2944,O
2944,0,130,11,2944,president,president,15558,15567,NOUN,NN,pobj,2941,O


In [14]:
import pandas as pd
df = pd.read_table('/content/test_booknlp/sample_babylon.entities')

In [53]:
df_verb = pd.read_table('/content/test_booknlp/sample_babylon.supersense')

In [54]:
df_verb.head()

,start_token,end_token,supersense_category,text
0,1,1,noun.person,envoys
1,2,2,verb.motion,come
2,5,6,noun.object,giant space
3,7,7,noun.artifact,station
4,10,10,noun.communication,pilot


In [15]:
df.head()

,COREF,start_token,end_token,prop,cat,text
0,46,0,1,NOM,PER,Alien envoys
1,47,4,7,NOM,FAC,the giant space station
2,48,25,32,NOM,PER,a multitude of humans and non - humans
3,49,28,28,NOM,PER,humans
4,50,30,32,NOM,PER,non - humans


In [19]:
text_values = df.text.values
ref_id = df.COREF.values 
cat = df.cat.values
start_token_id = df.start_token.values
extracted = []

In [ ]:
sentences

In [92]:
lenghts = [len(x.split()) for x in sentences]

In [ ]:
len(sentences[0].split())

In [ ]:
lenghts

In [94]:
for i in range(len(lenghts)):
  if i!=0:
    lenghts[i] = lenghts[i]+lenghts[i-1]

In [ ]:
lenghts

In [80]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [97]:
for j in range(len(sentences)):
  check = False
  identities = set()
  for i in range(len(text_values)):
    if start_token_id[i]<lenghts[j-1] and i!=0:
      continue
    if start_token_id[i]>lenghts[j]:
      break
    if text_values[i] in sentences[j] and cat[i]=='PER':
      identities.add((text_values[i], ref_id[i]))
  if len(identities)==2:
    print('###################################')
    print(f' \t Sentence {j}')
    print('###################################')
    print(identities)
    check = True
  for idx, row in df_verb.iterrows():
    if "verb.social" in row.supersense_category and check:
      if row.start_token>lenghts[j]:
        break  
      if row.start_token<lenghts[j-1] and i!=0:
        continue
      print(f"Verb: {row.text} and type: {row.supersense_category}")
      print(sentences[j])
      ss = SentimentIntensityAnalyzer().polarity_scores(sentences[j])
      del ss['compound']
      print(max(ss, key=ss.get), ss[max(ss, key=ss.get)])
      print()

###################################
 	 Sentence 22
###################################
{('Commander Sinclair', 19), ('Kosh', 28)}
Verb: allowing and type: verb.social
 As she conducts the scan, Lyta sees what Kosh saw as he boarded the station: Commander Sinclair welcoming him aboard, shaking his hand--and poisoning him
neu 0.689

###################################
 	 Sentence 44
###################################
{('her', 29), ('he', 100)}
Verb: doing and type: verb.social
 Upon further investigation in Varner's quarters, Garibaldi learns that Varner had been smuggling illegal items between systems, and that he most recently had gone to the Antares sector to acquire a changeling net--a device that can make an individual appear to look like somebody else
neu 0.817

Verb: tries and type: verb.social
 Upon further investigation in Varner's quarters, Garibaldi learns that Varner had been smuggling illegal items between systems, and that he most recently had gone to the Antares sector to

In [72]:
if "begins" in df_verb.text.values:
  print(df_verb.supersense_category.values[list(df_verb.text.values).index("help")])

verb.social


In [61]:
type(df_verb.text.values)

numpy.ndarray